# P4

In [21]:
%load_ext autoreload

In [2]:
%autoreload 2
# from src import eliminacion_gaussiana, gauss_jordan
# las funciones se encuentran disponibles como referencia

In [23]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)
logging.info(datetime.now())

[08-14 08:54:11][INFO] 2024-08-14 08:54:11.179421


In [25]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [27]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[08-14 08:54:12][INFO] 2024-08-14 08:54:12.574446
[08-14 08:54:12][INFO] 2024-08-14 08:54:12.577742
[08-14 08:54:12][INFO] 2024-08-14 08:54:12.579680
[08-14 08:54:12][INFO] 2024-08-14 08:54:12.582693
[08-14 08:54:12][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[08-14 08:54:12][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[08-14 08:54:12][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[08-14 08:54:12][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

In [55]:
# ####################################################################
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A)
    assert A.shape[0] == A.shape[1] - 1, "La matriz A debe ser de tamaño n-by-n."
    n = A.shape[0]
    Ab = np.hstack([A, np.eye(n)])
    
    for i in range(n):
        p = None
        for pi in range(i, n):
            if Ab[pi, i] != 0:
                if p is None or abs(Ab[pi, i]) > abs(Ab[p, i]):
                    p = pi

        if p is None:
            raise ValueError("No existe solución única, la matriz es singular.")
        
        if p != i:
            Ab[[i, p]] = Ab[[p, i]]
        
        Ab[i, :] /= Ab[i, i]
        
        for j in range(n):
            if i != j:
                Ab[j, :] -= Ab[j, i] * Ab[i, :]

    A_inv = Ab[:, n:]
    return A_inv

## Ejemplos 
Compruebe su función con respecto a los siguientes ejemplos:
* Ejemplo 1

In [57]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

AssertionError: La matriz A debe ser de tamaño n-by-n.

* Ejemplo 2

In [8]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

# Ejercicios
* Ejercicio 1

In [49]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

AssertionError: La matriz A debe ser de tamaño n-by-(n+1).

* Ejercicio 2

In [10]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

* Ejercicio 3

In [11]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)